## Demonstrate interactions with traitentity relations using omnikeeper

In [14]:
import os
import omnikeeper_client as okc
from omnikeeper_client import TraitDefinition, TraitAttributeDefinition, TraitRelationDefinition, ATTRIBUTETYPE_TEXT, ATTRIBUTETYPE_INTEGER

As usual create the omnikeeper client

In [15]:
okapiclient = okc.OkApiClient(
    backend_url=os.getenv('OMNIKEEPER_URL'),
    client_id=os.getenv('OMNIKEEPER_AUTH_CLIENTID'),
    username=os.getenv('OMNIKEEPER_AUTH_USERNAME'),
    password=os.getenv('OMNIKEEPER_AUTH_PASSWORD'),
)

In this example we will work with two traits host and interface and we will define relations using okc public functions. 

First lets define the trait names and layer.

In [16]:
host_trait_id = "python_client_demo.host"
interface_trait_id = "python_client_demo.interface"
layer_id = "traitrelations_test"

Create the layer, traits and trait entities

In [17]:
# create layer
ret = okc.create_layer(okapiclient, layer_id, "layer used for relation tests", okc.hex_string_to_rgb_color("#FF0000"))
assert(ret)

# create traits
ret = is_host_trait_created = okc.upsert_trait(okapiclient, TraitDefinition(host_trait_id, [
        TraitAttributeDefinition("id", "host.id", ATTRIBUTETYPE_INTEGER),
      ], optional_relations = [
        TraitRelationDefinition(identifier = "has_interface", predicate_id = "has_interface", direction_forward=True)
      ]))
assert(ret)

ret = okc.upsert_trait(okapiclient, TraitDefinition(interface_trait_id, [
        TraitAttributeDefinition("id", "interface.id", ATTRIBUTETYPE_INTEGER),
      ], optional_relations = [
        TraitRelationDefinition(identifier = "belongs_to_host", predicate_id = "has_interface", direction_forward=False)
      ]))
assert(ret)

# insert hosts (attributes only)
hosts = [
    {"ciid": "136DA01F-9ABD-4D9D-80C7-02AF85C822A8", "id": 1},
    {"ciid": "236DA01F-9ABD-4D9D-80C7-02AF85C822A8", "id": 2},
]
ret = okc.bulk_replace_trait_entities(okapiclient, host_trait_id, hosts, layer_id, layer_id)
assert(ret)

# insert interfaces (attributes only)
interfaces = [
    {"ciid": "736DA01F-9ABD-4D9D-80C7-02AF85C822A8", "id": 1},
    {"ciid": "836DA01F-9ABD-4D9D-80C7-02AF85C822A8", "id": 2},
    {"ciid": "936DA01F-9ABD-4D9D-80C7-02AF85C822A8", "id": 3}
]
ret = okc.bulk_replace_trait_entities(okapiclient, interface_trait_id, interfaces, layer_id, layer_id)
assert(ret)

# insert relations between hosts and services
ret = okc.set_traitentity_relations(okapiclient, host_trait_id, "has_interface", hosts[0]["ciid"], [interfaces[0]["ciid"], interfaces[1]["ciid"]], layer_id, layer_id)
assert(ret)
ret = okc.set_traitentity_relations(okapiclient, host_trait_id, "has_interface", hosts[1]["ciid"], [interfaces[2]["ciid"]], layer_id, layer_id)
assert(ret)


Create the host and interface CIs

Fetch host and interface trait entities (attributes only)

In [18]:
# hosts_list = [
#     {"id": 1},
#     {"id": 2},
#     {"id": 3},
# ]
# okc.bulk_replace_trait_entities_by_filter(okapiclient, trait_id=host_trait_id, input=hosts_list, id_attributes=["id"], id_relations=[], write_layer=layer_id, filter={})

Create Interface traitentities

In [19]:
# interfaces_list = [
#     {"id": 1},
#     {"id": 2}, 
#     {"id": 3}
# ]
# okc.bulk_replace_trait_entities_by_filter(okapiclient, trait_id=interface_trait_id, input=interfaces_list, id_attributes=["id"], id_relations=[], write_layer=layer_id, filter={})

Fetch the created trait entities

In [20]:
hosts = okc.get_all_traitentities(okapiclient, trait_id=host_trait_id, layers=[layer_id])
hosts

[{'ciid': '136da01f-9abd-4d9d-80c7-02af85c822a8', 'id': 1},
 {'ciid': '236da01f-9abd-4d9d-80c7-02af85c822a8', 'id': 2}]

In [21]:
interfaces = okc.get_all_traitentities(okapiclient, trait_id=interface_trait_id, layers=[layer_id])
interfaces

[{'ciid': '736da01f-9abd-4d9d-80c7-02af85c822a8', 'id': 1},
 {'ciid': '836da01f-9abd-4d9d-80c7-02af85c822a8', 'id': 2},
 {'ciid': '936da01f-9abd-4d9d-80c7-02af85c822a8', 'id': 3}]

Fetch belongs_to_host traitrelations. In the result interfaces will be as base_cis and hosts as related cis

In [22]:
has_host_relations = okc.get_all_traitentity_relations(okapiclient, trait_id=interface_trait_id, traitrelation_id="belongs_to_host", layers=[layer_id])
has_host_relations

[{'base_ciid': '736da01f-9abd-4d9d-80c7-02af85c822a8',
  'related_ciids': ['136da01f-9abd-4d9d-80c7-02af85c822a8']},
 {'base_ciid': '836da01f-9abd-4d9d-80c7-02af85c822a8',
  'related_ciids': ['136da01f-9abd-4d9d-80c7-02af85c822a8']},
 {'base_ciid': '936da01f-9abd-4d9d-80c7-02af85c822a8',
  'related_ciids': ['236da01f-9abd-4d9d-80c7-02af85c822a8']}]

Fetch has_interface traitrelations for all hosts

In [23]:
has_interface_relations = okc.get_all_traitentity_relations(okapiclient, trait_id=host_trait_id, traitrelation_id="has_interface", layers=[layer_id])
has_interface_relations

[{'base_ciid': '136da01f-9abd-4d9d-80c7-02af85c822a8',
  'related_ciids': ['736da01f-9abd-4d9d-80c7-02af85c822a8',
   '836da01f-9abd-4d9d-80c7-02af85c822a8']},
 {'base_ciid': '236da01f-9abd-4d9d-80c7-02af85c822a8',
  'related_ciids': ['936da01f-9abd-4d9d-80c7-02af85c822a8']}]

Merge hosts with related interfaces

In [24]:
interface_dict = {interface['ciid']: interface for interface in interfaces}

hosts_with_relations = []
for host in hosts:
    merged_dict = host.copy()
    merged_dict['related_ciis'] = []
    for relation in has_interface_relations:
        if host['ciid'] == relation['base_ciid']:
            for related_ciid in relation['related_ciids']:
                if related_ciid in interface_dict:
                    merged_dict['related_ciis'].append(interface_dict[related_ciid])
    hosts_with_relations.append(merged_dict)

hosts_with_relations

[{'ciid': '136da01f-9abd-4d9d-80c7-02af85c822a8',
  'id': 1,
  'related_ciis': [{'ciid': '736da01f-9abd-4d9d-80c7-02af85c822a8', 'id': 1},
   {'ciid': '836da01f-9abd-4d9d-80c7-02af85c822a8', 'id': 2}]},
 {'ciid': '236da01f-9abd-4d9d-80c7-02af85c822a8',
  'id': 2,
  'related_ciis': [{'ciid': '936da01f-9abd-4d9d-80c7-02af85c822a8', 'id': 3}]}]

The same can be done for interfaces to fetch its relations

In [25]:
host_dict = {host['ciid']: host for host in hosts}

interfaces_with_relations = []
for interface in interfaces:
    merged_dict = interface.copy()
    merged_dict['related_ciis'] = []
    for relation in has_host_relations:
        if interface['ciid'] == relation['base_ciid']:
            for related_ciid in relation['related_ciids']:
                if related_ciid in host_dict:
                    merged_dict['related_ciis'].append(host_dict[related_ciid])
    interfaces_with_relations.append(merged_dict)

interfaces_with_relations

[{'ciid': '736da01f-9abd-4d9d-80c7-02af85c822a8',
  'id': 1,
  'related_ciis': [{'ciid': '136da01f-9abd-4d9d-80c7-02af85c822a8', 'id': 1}]},
 {'ciid': '836da01f-9abd-4d9d-80c7-02af85c822a8',
  'id': 2,
  'related_ciis': [{'ciid': '136da01f-9abd-4d9d-80c7-02af85c822a8', 'id': 1}]},
 {'ciid': '936da01f-9abd-4d9d-80c7-02af85c822a8',
  'id': 3,
  'related_ciis': [{'ciid': '236da01f-9abd-4d9d-80c7-02af85c822a8', 'id': 2}]}]

Use bulk replace to completely change the relations

In [26]:
okc.bulk_replace_traitentity_relations(okapiclient, host_trait_id, "has_interface", 
                                       input=[{'baseCIID': '236da01f-9abd-4d9d-80c7-02af85c822a8', 'relatedCIIDs': ['736da01f-9abd-4d9d-80c7-02af85c822a8', '936da01f-9abd-4d9d-80c7-02af85c822a8']}], 
                                       relevant_ciids=['136DA01F-9ABD-4D9D-80C7-02AF85C822A8', '236DA01F-9ABD-4D9D-80C7-02AF85C822A8'], write_layer=layer_id)
has_interface_relations = okc.get_all_traitentity_relations(okapiclient, trait_id=host_trait_id, traitrelation_id="has_interface", layers=[layer_id])
has_interface_relations

GraphQLError: Unknown type 'GuidGraphType'.

GraphQL request:2:124
2 |         mutation($readLayers: [String]!, $writeLayer: String!, $input: [TE_Upser
  | t_Relations_Only_Input]!, $relevantCIIDs: [GuidGraphType]!) {
  |                                            ^

In [ ]:
# cleanup
okc.delete_trait(okapiclient, host_trait_id)
okc.delete_trait(okapiclient, interface_trait_id)
okc.truncate_layer(okapiclient, layer_id)

True